In [1]:
# hard parameter sharing
import numpy as np
import torch

In [2]:
import pandas as pd
from datasets import load_abundance_data, load_markers_data
from baseline.training_functions import tune_VAE, tune_SAE, tune_DAE, tune_RF, tune_SVM, tune_FFNN
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
dfs=load_abundance_data()
# markers=load_markers_data()

In [4]:
disease_map=pd.DataFrame( ['Zeller_fecal_colorectal_cancer',
                         'Quin_gut_liver_cirrhosis',
                         'metahit',
                         't2dmeta_long',
                         'WT2D',
                         'Chatelier_gut_obesity', 
                          't2dmeta_short'], 
             ['Colorectal', 'Cirrhosis', 'IBD', 'C-T2D', 'EW-T2D', 'Obesity', 'C-T2D'] 
            ).reset_index()

disease_map.columns = ['clean', 'raw']
cols = []#['disease']
[cols.append( df.columns[df.columns.str.contains('k__')] ) for df in dfs]
all_dfs = pd.concat( [dfs[i][cols[i]].astype(float)# if i > 0
                      #else dfs[i][cols[i]] 
                      for i in range(len(dfs))], axis = 1).fillna(0)
all_dfs = all_dfs.T.groupby(['dataset_name']).sum().T
all_dfs['disease'] = pd.concat( [df['disease'] for df in dfs] )

all_dfs['dataset'] = all_dfs.index.str.split('.').str[0]
all_dfs = all_dfs[['dataset', 'disease']+list(all_dfs.columns[:-2])]
all_dfs = all_dfs.reset_index(drop=True)
all_dfs = all_dfs.fillna(0)
all_dfs=disease_map.merge(all_dfs, left_on='raw', right_on='dataset').drop(['raw', 'dataset'], axis=1)
all_dfs.columns=['dataset'] + list(all_dfs.columns[1:])

In [5]:
all_dfs.head()

,dataset,disease,k__Archaea|p__Euryarchaeota|c__Archaeoglobi|o__Archaeoglobales|f__Archaeoglobaceae|g__Archaeoglobaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcus|s__Methanocaldococcus_unclassified,k__Bacteria|p__Acidobacteria|c__Acidobacteriia|o__Acidobacteriales|f__Acidobacteriaceae|g__Acidobacteriaceae_unclassified,...,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Nakaseomyces|s__Candida_glabrata,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Naumovozyma|s__Naumovozyma_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomyces|s__Saccharomyces_cerevisiae,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomycetaceae_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_albicans,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_tropicalis,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_pombe,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_unclassified,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Eukaryota_noname|g__Entamoeba|s__Entamoeba_dispar,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Hexamitidae|g__Giardia|s__Giardia_intestinalis
0,Colorectal,n,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Colorectal,n,0.0,0.0,0.14833,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02143,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Colorectal,n,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Colorectal,cancer,0.0,0.0,0.04584,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Colorectal,n,0.0,0.0,5.88021,0.31221,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
datasets = [all_dfs.loc[all_dfs.dataset==a] for a in ['Obesity', 'Colorectal', 'IBD', 'EW-T2D'] ]

In [7]:
all_dfs.dataset.value_counts()

C-T2D         344
Obesity       253
Cirrhosis     232
Colorectal    121
IBD           110
EW-T2D         96
Name: dataset, dtype: int64

In [8]:
len( datasets )

4

## will start by doing param sharing through the autoencoders, as that is easier, and easily translates into an FFNN model

In [9]:
!ls

EDA.ipynb                  ffnn_summary.csv
Make_Result_Plots.ipynb    figures
Midpoint_Report.pdf        gia2105_ML4FG_Proposal.pdf
Run_Baseline_Models.ipynb  hard_param_sharing.ipynb
__pycache__                marker_pcoa.png
baseline                   results
checkpoint_dir             rf_summary.csv
dae_summary.csv            sae_summary.csv
data                       svm_summary.csv
datasets.py                vae_summary.csv


In [10]:
from baseline.training_functions import make_split

In [11]:
import imp
import baseline.training_functions as btf

In [12]:
datasets[0].head()

,dataset,disease,k__Archaea|p__Euryarchaeota|c__Archaeoglobi|o__Archaeoglobales|f__Archaeoglobaceae|g__Archaeoglobaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcus|s__Methanocaldococcus_unclassified,k__Bacteria|p__Acidobacteria|c__Acidobacteriia|o__Acidobacteriales|f__Acidobacteriaceae|g__Acidobacteriaceae_unclassified,...,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Nakaseomyces|s__Candida_glabrata,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Naumovozyma|s__Naumovozyma_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomyces|s__Saccharomyces_cerevisiae,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomycetaceae_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_albicans,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_tropicalis,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_pombe,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_unclassified,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Eukaryota_noname|g__Entamoeba|s__Entamoeba_dispar,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Hexamitidae|g__Giardia|s__Giardia_intestinalis
830,Obesity,obesity,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831,Obesity,leaness,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,Obesity,leaness,0.0,0.0,3.71449,0.29400,0.0,0.01828,0.0,0.00000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
833,Obesity,obesity,0.0,0.0,0.38160,0.04919,0.0,0.00000,0.0,0.00000,...,0.0,0.0,0.00031,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834,Obesity,obesity,0.0,0.0,0.04493,0.01753,0.0,0.00000,0.0,0.00979,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#doing train/valid/test splits across all datasets
splits = [make_split(df) for df in datasets]
splits = [(make_split(a[0]), a[1]) for a in splits ]

trains = [s[0][0] for s in splits]
vals = [s[0][1] for s in splits]
tests = [s[1] for s in splits]

In [14]:
all_train = pd.concat(trains, axis=0)
all_val = pd.concat(vals, axis=0)
all_tests = pd.concat(tests, axis=0) # don't really need to combine the test things together

In [15]:
from baseline.pytorch_models import LitDAE

In [16]:
all_train.head()

,dataset,disease,k__Archaea|p__Euryarchaeota|c__Archaeoglobi|o__Archaeoglobales|f__Archaeoglobaceae|g__Archaeoglobaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcus|s__Methanocaldococcus_unclassified,k__Bacteria|p__Acidobacteria|c__Acidobacteriia|o__Acidobacteriales|f__Acidobacteriaceae|g__Acidobacteriaceae_unclassified,...,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Nakaseomyces|s__Candida_glabrata,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Naumovozyma|s__Naumovozyma_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomyces|s__Saccharomyces_cerevisiae,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomycetaceae_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_albicans,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_tropicalis,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_pombe,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_unclassified,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Eukaryota_noname|g__Entamoeba|s__Entamoeba_dispar,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Hexamitidae|g__Giardia|s__Giardia_intestinalis
1080,Obesity,leaness,0.0,0.0,0.07109,0.00737,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,Obesity,obesity,0.0,0.0,0.08104,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,Obesity,leaness,0.0,0.0,0.07591,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1062,Obesity,leaness,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846,Obesity,leaness,0.0,0.0,0.07258,0.00227,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
all_train.shape

(370, 773)

In [18]:
all_val.shape

(93, 773)

In [19]:
all_tests.shape

(117, 773)

In [20]:
DAE_trainer = LitDAE(all_train, 
                     all_val)

In [23]:
from baseline.training_functions import *

In [25]:
lightning = LitDAE(all_train, all_val)

#setr up the trainer/logger/callbacks
checkpoint_callback=ModelCheckpoint(
                dirpath = 'checkpoint_dir',
                save_top_k=1,
                verbose=False,
                monitor='val_loss',
                mode='min'
                )

tube_logger = TestTubeLogger('checkpoint_dir', 
                            name='test_tube_logger')

trainer = pl.Trainer(max_epochs = 500,
                     logger=tube_logger,
                     progress_bar_refresh_rate=0,
                     weights_summary=None,
                     check_val_every_n_epoch=1,
                     checkpoint_callback=checkpoint_callback,
                    callbacks=[EarlyStopping(monitor='val_loss', 
                                            patience=20)]) #the patience of 20 is mentioned in the DeepMicro paper

#run training
trainer.fit(lightning)

#load model from best-performing epoch
lightning.load_state_dict(torch.load(checkpoint_callback.best_model_path )['state_dict'])

#delete the files once we're done with them -- no need to store things unnecessarily
os.system('rm '+ checkpoint_callback.best_model_path)
os.system('rm -R checkpoint_dir/test_tube_logger/*')

#set model to eval()
lightning=lightning.eval()



GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [26]:

lightning

LitDAE(
  (model): DAE(
    (encoder): Sequential(
      (0): Linear(in_features=771, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
    )
    (decoder): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=771, bias=True)
      (3): ReLU()
      (4): Softmax(dim=None)
    )
  )
  (loss_func): MSELoss()
)

In [28]:
def build_sharing_encoder_eval_func(train,
                                    valid,
                                    test_df,
                                    all_trains, 
                                    all_valids,
                                    model_name = 'DAE', 
                                    is_marker=False
                                    ):
    def eval_func(hyperparams):
        """this function runs a complete train/valid/testing loop
        for an encoder model, given the specified hyperparameters, 
        it returns the AUC on the test set

        Defining it inside the tuning function to avoid having to manually pass the train_df and valid_df,
                Ax tuning doesn't allow that
        """
        #Initialize the model
        if model_name=='DAE':
            lightning = LitDAE(all_trains, 
                               all_valids, 
                               layer_1_dim = hyperparams['layer_1_size'],
                               layer_2_dim = hyperparams['layer_2_size'],
                               learning_rate = .001,
                               batch_size=50, 
                               is_marker=is_marker
                              )

        #setr up the trainer/logger/callbacks
        checkpoint_callback=ModelCheckpoint(
                        dirpath = 'checkpoint_dir',
                        save_top_k=1,
                        verbose=False,
                        monitor='val_loss',
                        mode='min'
                        )

        tube_logger = TestTubeLogger('checkpoint_dir', 
                                    name='test_tube_logger')

        trainer = pl.Trainer(max_epochs = 500,
                             logger=tube_logger,
                             progress_bar_refresh_rate=0,
                             weights_summary=None,
                             check_val_every_n_epoch=1,
                             checkpoint_callback=checkpoint_callback,
                            callbacks=[EarlyStopping(monitor='val_loss', 
                                                    patience=20)]) #the patience of 20 is mentioned in the DeepMicro paper

        #run training
        trainer.fit(lightning)

        #load model from best-performing epoch
        lightning.load_state_dict(torch.load(checkpoint_callback.best_model_path )['state_dict'])
        
        #delete the files once we're done with them -- no need to store things unnecessarily
        os.system('rm '+ checkpoint_callback.best_model_path)
        os.system('rm -R checkpoint_dir/test_tube_logger/*')
        
        #set model to eval()
        lightning=lightning.eval()

        train_ = MicroDataset(train, is_marker=is_marker)
        val_ = MicroDataset(valid, is_marker=is_marker)        
        
        # Fit the final predictions on just the dataset of interest
        X = np.vstack([lightning.model.encode(train_.matrix).detach().numpy(), 
                       lightning.model.encode(val_.matrix).detach().numpy()])

        y = np.hstack([train_.y.detach().numpy(),
                       val_.y.detach().numpy()] )

        #set up classifier model tuning
        if hyperparams['classifier_model']=='rf':
            eval_model=rf_eval
            eval_params = rf_parameters
        elif hyperparams['classifier_model']=='svm':
            eval_model=svm_eval
            eval_params=svm_parameters

        #tune the best classifier model, using 5-fold CV ==> this also employs AX under the hood
        best_parameters=eval_model(X, y, eval_params)

        # make a classifier with teh best hyperparams
        if hyperparams['classifier_model']=='rf':
            best_classifier=RandomForestClassifier()
            [setattr(best_classifier, a, q) for a,q in best_parameters.items()]

        elif hyperparams['classifier_model']=='svm':
            best_classifier=SVC(probability=True)
            [setattr(best_classifier, a, np.power(2., q)) for a,q in best_parameters.items()]

        # fit it on the full train, valid data
        best_classifier.fit(X, y)

        # Get AUC for the full setup on the test set
        test_dataset=MicroDataset(test_df, is_marker=is_marker)
        test_preds=best_classifier.predict_proba( lightning.model.encode(test_dataset.matrix).detach().numpy() )
        test_roc = roc_curve( test_dataset.y.detach().numpy(), test_preds[:,1] )
        auc_val=auc(test_roc[0], test_roc[1])
        if math.isnan(auc_val):
            auc_val=0
        return(auc_val) 
    
    return(eval_func)

In [47]:
def tune_shared_DAE(datasets, 
                    DAE_parameters=DAE_parameters, 
                    is_marker=False, 
                    seed=0, 
                    total_trials=8):
    #set a seed so that splits are the same across different model tests
    np.random.seed(seed)
    #make data splits
    #doing train/valid/test splits across all datasets
    splits = [make_split(df) for df in datasets]
    splits = [(make_split(a[0]), a[1]) for a in splits ]

    trains = pd.concat( [s[0][0] for s in splits], axis=0 ).reset_index(drop=True)
    vals = pd.concat( [s[0][1] for s in splits], axis=0 ).reset_index(drop=True)
    tests = pd.concat( [s[1] for s in splits], axis=0 ).reset_index(drop=True)
    
#     train_df, test_df = make_split(dataset)
#     train_df, valid_df = make_split(train_df)
    
    eval_func = build_sharing_encoder_eval_func(splits[0][0][0], 
                                                splits[0][0][1], 
                                                splits[0][1],
                                                all_trains=trains, 
                                                all_valids=vals,
                                                model_name = 'DAE', 
                                                is_marker=is_marker
                                                )

    #optimize hyperparapeter with Facebook's ax tuning 
    best_parameters, best_values, experiment, model = optimize(
        parameters = DAE_parameters,
        evaluation_function=eval_func, 
        total_trials = total_trials, 
        minimize=False)
    
    data_name=splits[0][0][0].index[0] 
    if is_marker==False:
        data_type='Abundance'
    else:
        data_type='Marker'
    
    return(['DAE', data_type, data_name, best_parameters, best_values])



In [48]:
DAE_param_share_out = tune_shared_DAE(datasets,
                                      DAE_parameters,
                                      is_marker=False)


[INFO 04-12 17:53:53] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 17:53:53] ax.service.managed_loop: Started full optimization with 8 steps.
[INFO 04-12 17:53:53] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-12 17:54:10] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 17:54:10] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 17:54:10] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 17:54:16] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 17:54:17] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-12 17:54:23] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-12 17:54:29] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 17:54:33] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 17:54:34] ax.s

[INFO 04-12 17:59:09] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 17:59:09] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-12 17:59:10] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-12 17:59:10] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-12 17:59:10] ax.service.managed_loop: Running optimization trial 10...


In [49]:
DAE_param_share_out

['DAE',
 'Abundance',
 991,
 {'layer_1_size': 64, 'layer_2_size': 32, 'classifier_model': 'svm'},
 ({'objective': 0.7249134948096885}, {'objective': {'objective': 0.0}})]

In [59]:
DAE_param_share_out_1

['DAE',
 'Abundance',
 829,
 {'layer_1_size': 1024, 'layer_2_size': 512, 'classifier_model': 'rf'},
 ({'objective': 0.5757575757575757}, {'objective': {'objective': 0.0}})]

In [60]:
datasets[::-1][0]

,dataset,disease,k__Archaea|p__Euryarchaeota|c__Archaeoglobi|o__Archaeoglobales|f__Archaeoglobaceae|g__Archaeoglobaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcaceae_unclassified,k__Archaea|p__Euryarchaeota|c__Methanococci|o__Methanococcales|f__Methanocaldococcaceae|g__Methanocaldococcus|s__Methanocaldococcus_unclassified,k__Bacteria|p__Acidobacteria|c__Acidobacteriia|o__Acidobacteriales|f__Acidobacteriaceae|g__Acidobacteriaceae_unclassified,...,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Nakaseomyces|s__Candida_glabrata,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Naumovozyma|s__Naumovozyma_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomyces|s__Saccharomyces_cerevisiae,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Saccharomycetaceae_unclassified,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_albicans,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetales_noname|g__Candida|s__Candida_tropicalis,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_pombe,k__Eukaryota|p__Ascomycota|c__Schizosaccharomycetes|o__Schizosaccharomycetales|f__Schizosaccharomycetaceae|g__Schizosaccharomyces|s__Schizosaccharomyces_unclassified,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Eukaryota_noname|g__Entamoeba|s__Entamoeba_dispar,k__Eukaryota|p__Eukaryota_noname|c__Eukaryota_noname|o__Eukaryota_noname|f__Hexamitidae|g__Giardia|s__Giardia_intestinalis
734,EW-T2D,n,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
735,EW-T2D,n,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
736,EW-T2D,n,0.0,0.0,3.83821,0.33097,0.35798,0.00787,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
737,EW-T2D,n,0.0,0.0,0.78534,0.00000,0.00000,0.00608,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
738,EW-T2D,n,0.0,0.0,9.11862,0.00000,0.00000,0.02806,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,EW-T2D,n,0.0,0.0,0.02598,0.00000,0.00000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
826,EW-T2D,n,0.0,0.0,2.78607,0.00000,0.00000,0.00862,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
827,EW-T2D,n,0.0,0.0,2.46789,0.00000,0.00000,0.00000,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
828,EW-T2D,n,0.0,0.0,6.72433,0.07156,0.00000,0.00654,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
DAE_param_share_out_1 = tune_shared_DAE(datasets[::-1],
                                        DAE_parameters,
                                        is_marker=False
                                       )



[INFO 04-12 18:45:08] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 18:45:08] ax.service.managed_loop: Started full optimization with 8 steps.
[INFO 04-12 18:45:08] ax.service.managed_loop: Running optimization trial 1...
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
[INFO 04-12 18:45:18] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-12 18:45:18] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 18:45:18] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 18:45:23] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 18:45:27] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-12 18:45:29] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-12 18:45:31] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 18:45:34] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 18:45:34] ax.s

[INFO 04-12 18:50:46] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-12 18:50:46] ax.service.managed_loop: Running optimization trial 1...
[INFO 04-12 18:50:51] ax.service.managed_loop: Running optimization trial 2...
[INFO 04-12 18:50:51] ax.service.managed_loop: Running optimization trial 3...
[INFO 04-12 18:50:54] ax.service.managed_loop: Running optimization trial 4...
[INFO 04-12 18:50:56] ax.service.managed_loop: Running optimization trial 5...
[INFO 04-12 18:51:02] ax.service.managed_loop: Running optimization trial 6...
[INFO 04-12 18:51:05] ax.service.managed_loop: Running optimization trial 7...
[INFO 04-12 18:51:06] ax.service.managed_loop: Running optimization trial 8...
[INFO 04-12 18:51:06] ax.service.managed_loop: Running optimization trial 9...
[INFO 04-12 18:51:09] ax.service.managed_loop: Running optimization trial 10...


In [ ]:
datasets[0]

In [24]:
class FFNN_sharing(torch.nn.Module):
    """
    a deep FFNN network with a hard parameter sharing implementation
    """
    def __init__(self, 
                 datasets, 
                 layer_sizes = [128, 64, 32], 
                 dropout = .2):
        super(FFNN, self).__init__()
        
        self.n_datasets = len(datasets) # assume all datasets have the same input size 
                                        # ==> is kind of necessary for this to work
        
        shared_layers = [ nn.Linear( dataset[0].n_taxa, layer_sizes[0]), 
                          nn.BatchNorm1d(layer_sizes[0]), 
                          nn.Dropout(dropout), 
                          nn.GELU()]
        
        for i in range(len(layer_sizes)-1): # share everything ex
            shared_layers += [ nn.Linear(layer_sizes[i], layer_sizes[i+1]), 
                               nn.BatchNorm1d(layer_sizes[i+1]), 
                               nn.Dropout(dropout), 
                               nn.GELU()]
            
        outer_layers = torch.nn.ModuleList( 
                                            [ nn.Linear(layer_sizes[-1], 2), nn.Softmax() ] * self.n_datasets 
                                            )
            # one outer layer for each dataset we have
        
            
        self.shared_layers = nn.Sequential(*linear_layers)
        
    def forward(self, x, idx):
        out = self.shared_layers(x.float())
        out = outer_layers[idx](out) #take the specified outer layer
        return(out)

In [ ]:
class LitFFNN(pl.LightningModule):
    def __init__(self, 
                 train_datasets,
                 valid_datasets, 
                 layer_1_dim = 128,
                 layer_2_dim = 64,
                 layer_3_dim = 32,
                 dropout = .2,
                 learning_rate=1e-3, 
                 batch_size=50, 
                 is_marker=False):
        super().__init__()
        self.n_datasets = len(train_datasets)
        
        self.train_datasets = [ MicroDataset(train_df, is_marker=is_marker) for train_df in train_datasets ]
        self.valid_datasets = [ MicroDataset(valid_df, is_marker=is_marker) for valid_df in valid_datasets       
        
        self.train_loaders = DataLoader(self.train_dataset, batch_size = batch_size, shuffle = True)
        self.valid_loaders = DataLoader(self.valid_dataset, batch_size = batch_size, shuffle = False)
        self.model = FFNN(self.train_dataset, 
                          layer_sizes = [layer_1_dim, layer_2_dim, layer_3_dim],
                          dropout = dropout)
        self.learning_rate = learning_rate
        
        # they use MSE for reconstruction loss
        self.loss_func = nn.MSELoss()
                        
        # the training approach is to iteratively pick a dataset's index for each forward step
                                # and then backprop
        self.rand_data_idx = np.random.randint(self.n_datasets)
        
        
    def train_dataloader(self):
        return(self.train_loader)
    
    def val_dataloader(self):
        return(self.valid_loader)
    
    def forward(self, x):
        x = self.model(x)
        return x

    def split_batch(self, batch):
        return batch[0], batch[1]

    def training_step(self, batch, batch_idx):
        #train on ability to reconstruct x
        x, y = self.split_batch(batch)
        y_hat = self(x)
        loss = self.loss_func(y_hat[:, 1], y.float())
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = self.split_batch(batch)
        y_hat = self(x)
        loss = self.loss_func(y_hat[:, 1], y.float())
        self.log('val_loss', loss)
        return {'val_loss':loss}
 
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [12]:
def tune_FFNN(dataset, 
             FFNN_parameters=FFNN_parameters, 
             is_marker=False, 
             seed=0, 
             total_trials=8):
    #set a seed so that splits are the same across different model tests
    np.random.seed(seed)
    #make data splits
    train_df, test_df = make_split(dataset)
    train_df, valid_df = make_split(train_df)
    
    def FFNN_eval(hyperparams):
        """this function runs a complete train/valid/testing loop
            for an SAE model, given the specified hyperparameters, 
            it returns the AUC on the test set
            
            Defining it inside the tuning function to avoid having to manually pass the train_df and valid_df,
                    Ax tuning doesn't allow that
            """
        #Initialize the model
        lightning = LitFFNN(train_df, 
                            valid_df, 
                            layer_1_dim = hyperparams['layer_1_size'],
                            layer_2_dim = hyperparams['layer_2_size'],
                            layer_3_dim = hyperparams['layer_3_size'],
                            learning_rate = hyperparams['learning_rate'],
                            batch_size=50, 
                            is_marker=is_marker,
                            dropout=hyperparams['dropout']
                            )

        #setr up the trainer/logger/callbacks
        checkpoint_callback=ModelCheckpoint(
                        dirpath = 'checkpoint_dir',
                        save_top_k=1,
                        verbose=False,
                        monitor='val_loss',
                        mode='min'
                        )

        tube_logger = TestTubeLogger('checkpoint_dir', 
                                    name='test_tube_logger')

        trainer = pl.Trainer(max_epochs = 500,
                             logger=tube_logger,
                             progress_bar_refresh_rate=0,
                             weights_summary=None,
                             check_val_every_n_epoch=1,
                             checkpoint_callback=checkpoint_callback,
                            callbacks=[EarlyStopping(monitor='val_loss', 
                                                    patience=20)]) #the patience of 20 is mentioned in the DeepMicro paper

        #run training
        trainer.fit(lightning)

        #load model from best-performing epoch
        lightning.load_state_dict(torch.load(checkpoint_callback.best_model_path )['state_dict'])
        
        #delete the files once we're done with them -- no need to store things unnecessarily
        #=====> would eat up LOTS of memory if we didn't do this
        os.system('rm '+ checkpoint_callback.best_model_path)
        os.system('rm -R checkpoint_dir/test_tube_logger/*')
        
        #set model to eval()
        lightning=lightning.eval()

        # Get AUC for the full setup on the test set
        test_dataset=MicroDataset(test_df, is_marker=is_marker)
        test_loader=DataLoader(test_dataset, shuffle=False, batch_size=50)
        
        test_preds=[]
        test_trues=[]
        
        for batch in test_loader:
            test_preds.append(lightning(batch[0]))
            test_trues.append(batch[1])
        
        
        test_preds=torch.cat(test_preds).detach().numpy()
        test_roc = roc_curve( torch.cat(test_trues).detach().numpy(), test_preds[:,1] )
        auc_val=auc(test_roc[0], test_roc[1])
        if math.isnan(auc_val):
            auc_val=0
        return(auc_val) 

    #optimize hyperparapeter with Facebook's ax tuning 
    best_parameters, best_values, experiment, model = optimize(
        parameters = FFNN_parameters,
        evaluation_function=FFNN_eval, 
        total_trials = total_trials, 
        minimize=False)
    
    data_name=dataset.index[0] 
    if is_marker==False:
        data_type='Abundance'
    else:
        data_type='Marker'
    
    return(['FFNN', data_type, data_name, best_parameters, best_values])



NameError: name 'FFNN_parameters' is not defined

In [ ]:
a